In [0]:
// Create database and schema
CREATE DATABASE IF NOT EXISTS MYDB;
CREATE SCHEMA IF NOT EXISTS MYDB.file_formats;


-- Create a storage integration object
CREATE or replace STORAGE INTEGRATION snow_azure_int
  TYPE = EXTERNAL_STAGE
  STORAGE_PROVIDER = AZURE
  ENABLED = TRUE
  AZURE_TENANT_ID = '3e39d6f7-8861-4d25-a8a5-4f2c5fe27dcd'
  STORAGE_ALLOWED_LOCATIONS = ('azure://azdfdemostorage.blob.core.windows.net/snowflake');


-- Describe integration object
DESC STORAGE INTEGRATION snow_azure_int;


// Create database and schema
CREATE DATABASE IF NOT EXISTS MYDB;
CREATE SCHEMA IF NOT EXISTS MYDB.file_formats;
CREATE SCHEMA IF NOT EXISTS MYDB.external_stages;

// Create file format object
CREATE OR REPLACE file format mydb.file_formats.csv_fileformat
    type = csv
    field_delimiter = '|'
    skip_header = 1
    empty_field_as_null = TRUE;    
    
// Create stage object with integration object & file format object
CREATE OR REPLACE STAGE mydb.external_stages.stg_azure_cont
    URL = 'azure://azdfdemostorage.blob.core.windows.net/snowflake'
    STORAGE_INTEGRATION = snow_azure_int
    FILE_FORMAT = mydb.file_formats.csv_fileformat ;


//Listing files under your azure containers
list @mydb.external_stages.stg_azure_cont;
  

Snowflake AWS

In [0]:
%sql
SELECT CURRENT_ACCOUNT();

SELECT CURRENT_ROLE();

// Create storage integration object
create or replace storage integration s3_int
  TYPE = EXTERNAL_STAGE
  STORAGE_PROVIDER = S3
  ENABLED = TRUE 
  STORAGE_AWS_ROLE_ARN = 'arn:aws:iam::905418375640:role/SnowflakeS3AccessRole'
  STORAGE_ALLOWED_LOCATIONS = ('s3://ramakrishna12')
  COMMENT = 'Integration with aws s3 buckets' ;


DESC integration s3_int;


// Create database and schema
CREATE DATABASE IF NOT EXISTS MYDB;
CREATE SCHEMA IF NOT EXISTS MYDB.file_formats;

// Create file format object
CREATE OR REPLACE file format mydb.file_formats.csv_fileformat
    type = csv
    field_delimiter = '|'
    skip_header = 1
    empty_field_as_null = TRUE;   


// Create stage object with integration object & file format object
CREATE OR REPLACE STAGE mydb.external_stages.aws_s3_csv
    URL = 's3://ramakrishna12'
    STORAGE_INTEGRATION = s3_int
    FILE_FORMAT = mydb.file_formats.csv_fileformat ;


//Listing files under your s3 buckets
list @mydb.external_stages.aws_s3_csv;


CREATE OR REPLACE TABLE mydb.public.employee_data (
    empid NUMBER,
    empname STRING,
    gender STRING,
    country STRING
);




COPY INTO mydb.public.employee_data
FROM @mydb.external_stages.aws_s3_csv 
FILE_FORMAT = (TYPE = CSV,  SKIP_HEADER = 1);


select * from mydb.public.employee_data




// Generate files and store them in the stage location
COPY INTO @mydb.external_stages.aws_s3_csv
FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.CUSTOMER;


//Listing files under my s3 bucket
LIST @mydb.external_stages.aws_s3_csv;


// Specifiy the filename in the copy command
COPY INTO @mydb.external_stages.aws_s3_csv/customer
FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.CUSTOMER;

// MAX_FILE_SIZE
COPY INTO @mydb.external_stages.aws_s3_csv/customer
FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.CUSTOMER
MAX_FILE_SIZE=2000000;


// Use OVERWRTIE=TRUE
// If we want to overwrite existing file we can set that to TRUE
COPY INTO @mydb.external_stages.aws_s3_csv/customer
FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.CUSTOMER
MAX_FILE_SIZE=2000000
OVERWRITE = TRUE;



//Listing files under my s3 bucket
LIST @mydb.external_stages.aws_s3_csv;

//generate single file
COPY INTO @mydb.external_stages.aws_s3_csv/Ramakrishna
FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.CUSTOMER
SINGLE = TRUE;


//detailed output
COPY INTO @mydb.external_stages.aws_s3_csv/cust_data
FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.CUSTOMER
DETAILED_OUTPUT = TRUE;


SHOW WAREHOUSES

-- Replace the placeholders with your actual connection details
USE DATABASE dbt;
USE SCHEMA snowdbt;

SELECT CURRENT_USER() AS current_user,
       CURRENT_ROLE() AS current_role,
       CURRENT_WAREHOUSE() AS current_warehouse,
       CURRENT_DATABASE() AS current_database,
       CURRENT_SCHEMA() AS current_schema;


Snowflake DBT

In [0]:
%sql
CREATE or replace SCHEMA L1_LANDING;

CREATE or replace SCHEMA L2_PROCESSING;

CREATE or replace SCHEMA snowdbt;

 

USE database dbt;

USE SCHEMA L1_LANDING;



CREATE TABLE IF NOT EXISTS Dates (

Date DATE NOT NULL,

Day VARCHAR(3) NULL,

Month VARCHAR(10) NULL,

Year VARCHAR(4) NULL,

Quarter INT NULL,

DayOfWeek VARCHAR(10) NULL,

WeekOfYear INT NULL,

Updated_at TIMESTAMP NULL,

PRIMARY KEY (Date));


INSERT INTO Dates (Date, Day, Month, Year, Quarter, DayOfWeek, WeekOfYear, Updated_at)
VALUES 
    ('2024-07-13', 'Thu', 'July', '2024', 3, 'Thursday', 28, CURRENT_TIMESTAMP),
    ('2024-07-14', 'Fri', 'July', '2024', 3, 'Friday', 28, CURRENT_TIMESTAMP),
    ('2024-07-15', 'Sat', 'July', '2024', 3, 'Saturday', 28, CURRENT_TIMESTAMP),
    ('2024-07-16', 'Sun', 'July', '2024', 3, 'Sunday', 29, CURRENT_TIMESTAMP),
    ('2024-07-17', 'Mon', 'July', '2024', 3, 'Monday', 29, CURRENT_TIMESTAMP);




CREATE TABLE IF NOT EXISTS customers (

CustomerID int null,

FirstName VARCHAR(50),

LastName VARCHAR(50),

Email VARCHAR(100),

Phone VARCHAR(100),

Address VARCHAR(100),

City VARCHAR(50),

State VARCHAR(2),

ZipCode VARCHAR(10),

Updated_at TIMESTAMP);


INSERT INTO customers (CustomerID, FirstName, LastName, Email, Phone, Address, City, State, ZipCode, Updated_at)
VALUES 
    (1, 'John', 'Doe', 'john.doe@example.com', '123-456-7890', '123 Main St', 'Anytown', 'NY', '12345', CURRENT_TIMESTAMP),
    (2, 'Jane', 'Smith', 'jane.smith@example.com', '987-654-3210', '456 Elm St', 'Otherville', 'CA', '54321', CURRENT_TIMESTAMP),
    (3, 'Michael', 'Johnson', 'michael.johnson@example.com', '555-555-5555', '789 Oak Ave', 'Sometown', 'TX', '67890', CURRENT_TIMESTAMP),
    (4, 'Emily', 'Brown', 'emily.brown@example.com', '111-222-3333', '321 Pine Ln', 'Anycity', 'FL', '98765', CURRENT_TIMESTAMP),
    (5, 'David', 'Wilson', 'david.wilson@example.com', '444-333-2222', '654 Cedar Rd', 'Anothercity', 'WA', '23456', CURRENT_TIMESTAMP);



CREATE TABLE IF NOT EXISTS Employees (

EmployeeID INT NOT NULL,

FirstName VARCHAR(100) NULL,

LastName VARCHAR(100) NULL,

Email VARCHAR(200) NULL,

JobTitle VARCHAR(100) NULL,

HireDate DATE NULL,

ManagerID INT NULL,

Address VARCHAR(200) NULL,

City VARCHAR(50) NULL,

State VARCHAR(50) NULL,

ZipCode VARCHAR(10) NULL,

Updated_at TIMESTAMP NULL,

PRIMARY KEY (EmployeeID));


INSERT INTO Employees (EmployeeID, FirstName, LastName, Email, JobTitle, HireDate, ManagerID, Address, City, State, ZipCode, Updated_at)
VALUES 
    (1, 'John', 'Smith', 'john.smith@example.com', 'Manager', '2020-01-15', NULL, '123 Main St', 'Anytown', 'NY', '12345', CURRENT_TIMESTAMP),
    (2, 'Jane', 'Doe', 'jane.doe@example.com', 'Engineer', '2021-03-20', 1, '456 Elm St', 'Otherville', 'CA', '54321', CURRENT_TIMESTAMP),
    (3, 'Michael', 'Johnson', 'michael.johnson@example.com', 'Analyst', '2019-07-10', 1, '789 Oak Ave', 'Sometown', 'TX', '67890', CURRENT_TIMESTAMP),
    (4, 'Emily', 'Brown', 'emily.brown@example.com', 'Developer', '2022-05-05', 2, '321 Pine Ln', 'Anycity', 'FL', '98765', CURRENT_TIMESTAMP),
    (5, 'David', 'Wilson', 'david.wilson@example.com', 'Designer', '2020-11-30', 1, '654 Cedar Rd', 'Anothercity', 'WA', '23456', CURRENT_TIMESTAMP);




CREATE TABLE IF NOT EXISTS Stores (

StoreID INT NOT NULL,

StoreName VARCHAR(100) NULL,

Address VARCHAR(200) NULL,

City VARCHAR(50) NULL,

State VARCHAR(50) NULL,

ZipCode VARCHAR(10) NULL,

Email VARCHAR(200) NULL,

Phone VARCHAR(50) NULL,

Updated_at TIMESTAMP NULL,

PRIMARY KEY (StoreID));

INSERT INTO Stores (StoreID, StoreName, Address, City, State, ZipCode, Email, Phone, Updated_at)
VALUES 
    (1, 'Main Street Store', '123 Main St', 'Anytown', 'NY', '12345', 'mainstreet@store.com', '555-1234', CURRENT_TIMESTAMP),
    (2, 'Elm Avenue Store', '456 Elm Ave', 'Otherville', 'CA', '54321', 'elmavenue@store.com', '555-5678', CURRENT_TIMESTAMP),
    (3, 'Oak Avenue Store', '789 Oak Ave', 'Sometown', 'TX', '67890', 'oakavenue@store.com', '555-9876', CURRENT_TIMESTAMP),
    (4, 'Pine Lane Store', '321 Pine Ln', 'Anycity', 'FL', '98765', 'pinelane@store.com', '555-4321', CURRENT_TIMESTAMP),
    (5, 'Cedar Road Store', '654 Cedar Rd', 'Anothercity', 'WA', '23456', 'cedarroad@store.com', '555-8765', CURRENT_TIMESTAMP);




CREATE TABLE IF NOT EXISTS Suppliers (

SupplierID INT NOT NULL,

SupplierName VARCHAR(100) NULL,

ContactPerson VARCHAR(100) NULL,

Email VARCHAR(200) NULL,

Phone VARCHAR(50) NULL,

Address VARCHAR(50) NULL,

City VARCHAR(50) NULL,

State VARCHAR(10) NULL,

ZipCode VARCHAR(10) NULL,

Updated_at TIMESTAMP NULL,

PRIMARY KEY (SupplierID));


INSERT INTO Suppliers (SupplierID, SupplierName, ContactPerson, Email, Phone, Address, City, State, ZipCode, Updated_at)
VALUES 
    (1, 'ABC Supplies', 'John Smith', 'john.smith@abcsupplies.com', '555-1234', '123 Elm St', 'Anytown', 'NY', '12345', CURRENT_TIMESTAMP),
    (2, 'XYZ Distributors', 'Jane Doe', 'jane.doe@xyzdist.com', '555-5678', '456 Oak Ave', 'Otherville', 'CA', '54321', CURRENT_TIMESTAMP),
    (3, '123 Products', 'Michael Johnson', 'michael.johnson@123products.com', '555-9876', '789 Maple Rd', 'Sometown', 'TX', '67890', CURRENT_TIMESTAMP),
    (4, 'Smith Enterprises', 'Emily Brown', 'emily.brown@smithent.com', '555-4321', '321 Pine Ln', 'Anycity', 'FL', '98765', CURRENT_TIMESTAMP),
    (5, 'Global Suppliers', 'David Wilson', 'david.wilson@globalsuppliers.com', '555-8765', '654 Cedar Rd', 'Anothercity', 'WA', '23456', CURRENT_TIMESTAMP);




CREATE TABLE IF NOT EXISTS Products (

ProductID INT NOT NULL,

Name VARCHAR(100) NULL,

Category VARCHAR(100) NULL,

RetailPrice DECIMAL(10,2) NULL,

SupplierPrice DECIMAL(10,2) NULL,

SupplierID INT NULL,

Updated_at TIMESTAMP NULL,

PRIMARY KEY (ProductID));

INSERT INTO Products (ProductID, Name, Category, RetailPrice, SupplierPrice, SupplierID, Updated_at)
VALUES 
    (1, 'Laptop', 'Electronics', 1200.00, 900.00, 1, CURRENT_TIMESTAMP),
    (2, 'Smartphone', 'Electronics', 800.00, 600.00, 2, CURRENT_TIMESTAMP),
    (3, 'Desk Chair', 'Furniture', 150.00, 120.00, 3, CURRENT_TIMESTAMP),
    (4, 'Coffee Maker', 'Appliances', 50.00, 35.00, 4, CURRENT_TIMESTAMP),
    (5, 'Running Shoes', 'Clothing', 80.00, 60.00, 5, CURRENT_TIMESTAMP);



CREATE TABLE IF NOT EXISTS OrderItems (

OrderItemID INT NOT NULL,

OrderID INT NULL,

ProductID INT NULL,

Quantity INT NULL,

UnitPrice DECIMAL(10,2) NULL,

Updated_at TIMESTAMP NULL,

PRIMARY KEY (OrderItemID));

INSERT INTO OrderItems (OrderItemID, OrderID, ProductID, Quantity, UnitPrice, Updated_at)
VALUES 
    (1, 101, 1, 2, 1200.00, CURRENT_TIMESTAMP),
    (2, 101, 2, 3, 800.00, CURRENT_TIMESTAMP),
    (3, 102, 3, 1, 150.00, CURRENT_TIMESTAMP),
    (4, 103, 4, 4, 50.00, CURRENT_TIMESTAMP),
    (5, 104, 5, 2, 80.00, CURRENT_TIMESTAMP);




CREATE TABLE IF NOT EXISTS Orders (

OrderID INT NOT NULL,

OrderDate DATE NULL,

CustomerID INT NULL,

EmployeeID INT NULL,

StoreID INT NULL,

Status VARCHAR(10) NULL,

Updated_at TIMESTAMP NULL,

PRIMARY KEY (OrderID));

INSERT INTO Orders (OrderID, OrderDate, CustomerID, EmployeeID, StoreID, Status, Updated_at)
VALUES 
    (101, '2024-07-13', 1, 1, 1, 'Pending', CURRENT_TIMESTAMP),
    (102, '2024-07-14', 2, 2, 2, 'Complete', CURRENT_TIMESTAMP),
    (103, '2024-07-15', 3, 3, 3, 'Pending', CURRENT_TIMESTAMP),
    (104, '2024-07-16', 4, 4, 4, 'Processing', CURRENT_TIMESTAMP),
    (105, '2024-07-17', 5, 5, 5, 'Complete', CURRENT_TIMESTAMP);


with customerorders as (
    select c.customerid, concat(c.firstname, ' ', c.lastname)as customername, count(o.orderid)as no_of_orders
    from l1_landing.customers c
    join l1_landing.orders o on c.customerid = o.customerid
    group by c.customerid, customername
    order by no_of_orders desc
)

select customerid, customername, no_of_orders from customerorders



--DBT Testing
select * from snowdbt.customerorders
